In [11]:
import requests
import json
import pprint

In [12]:
pip install elasticsearch

Note: you may need to restart the kernel to use updated packages.


In [13]:
from elasticsearch import Elasticsearch

In [25]:
es = Elasticsearch(["http://elasticsearch:9200"])
res = es.search(index="logs-training", body={"query": {"match_all": {}}})